<a href="https://colab.research.google.com/github/Joon2000/machinelearning/blob/main/5_3_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""트리의 앙상블"""

#앙상블 학습(ensemble learning) => 정형 데이터를 다루는데 가장 뛰어난 성능을 내는 알고리즘

#랜덤 포레스트
#결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다 => 각 결정 트리의 예측을 사용해 최종 예측을 함
#부트스트랩 샘블(bootstrap sample) => 훈련할 데이터를 랜덤하게 뽑음, 중복 혀용, 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다
#노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다 (분류 모델은 기본적으로 전체 특성 중 제곱근만큼의 특성을 찾음 => DecisionTree로 만듬 => Forest 만들기)
#회귀 모델은 전체 특성을 사용
#랜덤하게 샘플과 특성을 선택하기 때문에 과대적합을 막음
#분류는 다수결, 회귀는 평균으로 예측
#Decision Tree를 100개 정도 만듬

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH',]].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) #return_train_score을 통해 훈련세트에 대한 점수도 같이 반환, 매개변수 조작 안함
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대적합

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target) #특성을 랜덤하게 선택 => 일부 특성이 과도하게 중요해지는 것을 막음
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [6]:
#OOB(out of bag)샘플 => 부트슽랩 샘플에 포함되지 않고 남은 샘플
#OOB 샘플을 통해 rf를 평가할 수 있음 (검증 세트같은 역할)

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) # oob_score=True
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [8]:
# 엑스트라 트리(Extra Tree)
# 랜덤 트리와 비스스하지만 (특성을 랜덤하게 고름) 부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용하고 노드 분할 할 때 최적의 분할이 아닌 무작위 분할을 한다
# 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트 점수를 높인다

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#랜덤하게 노드를 분할하기 때문에 빠른계산 속도를 가짐

0.9974503966084433 0.8887848893166506


In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [11]:
#그래디언트 부스팅 (Gradient Boosting)
#깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완
#깊이가 3인 결정트리를 100개 사용
#경사 하강법과 비슷 => 결정 트리를 계속 추가하면서 가장 낮은 곳으로 이동
#학습률 매개변수로 속도를 조절 가능

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.8881086892152563 0.8720430147331015


In [13]:
#과대적합에 매우 강함
#학습률를 증가시키고 트리의 개수를 늘리면 성능이 향상됨
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#결정 트리: 100=>500개, 학습률: 0.1=>0.2

0.9464595437171814 0.8780082549788999


In [14]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
#subsample를 통해 훈련 세트의 비율을 정할 수 있음 => 각 tree마다 다른 샘플집단 사용 => 다양성 증가
#랜덤 포레스트보다 일부 특성에 집중
#훈련속도가 느림(n_jobs가 없음)

[0.15872278 0.68010884 0.16116839]


In [17]:
#히스토그램 기반 그레디언트 부스팅(Histogram-based Gradient Boosting)
#가장 인기가 많음
#훈련 데이터을 256개의 구간으로 나눔 => 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음 (데이의 구간별 통계적 정보를 미리 계산)
#256개의 구간 중 하나를 누락된 값을 위해 씀 => 누락된 특성을 전처리 할 필요가 없음
#트리의 개수를 n_estimators 대신해 부스팅 반복 횟수를 지정하는 max_iter를 사용

from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input,train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.9321723946453317 0.8801241948619236


In [19]:
#특성 중요도를 계산하기 위해 permutation_importance() 함수를 사용 => 특성을 하나씩 랜덤하게 섞어서 모델의 성능의 변화를 관찰해서 어떤 특성이 중요한지 계산
#사이킷런에서 제공하는 추정기 모델에 모두 사용 가능

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1) #n_repeats로 랜덤하게 섞을 횟수를 지정
print(result.importances_mean)
#[특성 중요도, 평균, 표준편차]

[0.08876275 0.23438522 0.08027708]


In [20]:
# 테스트 세트의 특성 중요도
result= permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [21]:
hgb.score(test_input, test_target)

0.8723076923076923

In [22]:
#XGBoost 라이브러리
#사이킷런의 cross_validate와 같이 쓸 수 있음

from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42) # tree_method='hist'
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [24]:
#LightGBM
#사이킷런의 cross_validate와 같이 쓸 수 있음

from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores=cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
